In [59]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy.dialects.postgresql import insert
import sqlalchemy
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("Sample - Superstore.csv", encoding = "latin1")

In [4]:
df.columns.values

array(['Row ID', 'Order ID', 'Order Date', 'OrderDate2', 'Ship Date',
       'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country',
       'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Revenue', 'Order_Date2'], dtype=object)

######  connecting to data warehouse

In [8]:
# Connection parameters
username = 'postgres'          
password = 'Best2410#'      
host = 'localhost'         
port = '5432'              
database = 'sales_warehouse' 

# Create engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


In [30]:
#ckecking tables
table_names = engine.table_names()
print(table_names)

['dim_date', 'fact_sales', 'dim_customer', 'dim_product']


###### Performing ETL

###  Extracting

In [10]:
df.shape

(9994, 24)

In [11]:
#checking for mssing values
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
OrderDate2       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
Revenue          0
Order_Date2      0
dtype: int64

No missing values in the dataset 

In [12]:
#Getting dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   OrderDate2     9994 non-null   object 
 4   Ship Date      9994 non-null   object 
 5   Ship Mode      9994 non-null   object 
 6   Customer ID    9994 non-null   object 
 7   Customer Name  9994 non-null   object 
 8   Segment        9994 non-null   object 
 9   Country        9994 non-null   object 
 10  City           9994 non-null   object 
 11  State          9994 non-null   object 
 12  Postal Code    9994 non-null   int64  
 13  Region         9994 non-null   object 
 14  Product ID     9994 non-null   object 
 15  Category       9994 non-null   object 
 16  Sub-Category   9994 non-null   object 
 17  Product Name   9994 non-null   object 
 18  Sales   

##### NOTE
* There seem to be some repeated columns (Order Date, OrderDate2 and Order_Date2). Checking to resolve it

In [15]:
df['Order Date'] is df["Order_Date2"]

False

In [16]:
df['Order Date'] is df['OrderDate2']

False

In [19]:
df['Order Date'].head()

0     11/8/2016
1     11/8/2016
2     6/12/2016
3    10/11/2015
4    10/11/2015
Name: Order Date, dtype: object

In [21]:
df['Order_Date2'].head()

0    2016-11
1    2016-11
2    2016-06
3    2015-10
4    2015-10
Name: Order_Date2, dtype: object

In [23]:
df['OrderDate2'].head()

0    2016-11
1    2016-11
2    2016-06
3    2015-10
4    2015-10
Name: OrderDate2, dtype: object

###### NOTE
* From the above, all columns are the same. So, I will drop Order_Date2 and OrderDate2

In [24]:
#dropping columns
df = df.drop(columns=['Order_Date2','OrderDate2'], axis = 1)

In [25]:
df.columns.values

array(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Revenue'], dtype=object)

In [48]:
df.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sales', 'quantity', 'discount', 'profit', 'revenue',
       'date_x', 'order_date_id', 'date_y', 'ship_date_id'],
      dtype='object')

In [78]:
df['order_date'].nunique()

1237

###  Transforming

In [37]:
# Transforming dim_date

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

all_dates = pd.concat([df['Order Date'], df['Ship Date']]).drop_duplicates().reset_index(drop=True)
dim_date = pd.DataFrame({'date': all_dates})
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['day'] = dim_date['date'].dt.day
dim_date['quarter'] = dim_date['date'].dt.quarter
dim_date['weekday'] = dim_date['date'].dt.day_name()
dim_date['date_id'] = range(1, len(dim_date)+1)

In [39]:
# Mapping dates to date_id for fact table
df = df.merge(dim_date[['date','date_id']], left_on='Order Date', right_on='date', how='left')
df.rename(columns={'date_id':'order_date_id'}, inplace=True)
df = df.merge(dim_date[['date','date_id']], left_on='Ship Date', right_on='date', how='left')
df.rename(columns={'date_id':'ship_date_id'}, inplace=True)


In [49]:
# Transforming dim_customer
dim_customer = df[['customer_id','customer_name','segment','country',
                   'city','state','postal_code','region']].drop_duplicates()

In [50]:
# Transforming dim_product
dim_product = df[['product_id','product_name',
                  'category','sub_category']].drop_duplicates()

In [79]:
#Transforming fact_sales
fact_sales = df[['order_id','order_date_id','ship_date_id','customer_id',
                 'product_id',
                 'sales','quantity','discount','profit','revenue']]

###  LOADING

######  Loading Into datawarehouse

In [80]:
def upsert_table(df, table_name, key_column, engine):
    """
    Safely inserts DataFrame rows into PostgreSQL using ON CONFLICT DO NOTHING.
    Avoids duplicate primary key errors.
    """
    df = df.drop_duplicates(subset=[key_column])

    metadata = MetaData()
    table = Table(table_name, metadata, autoload_with=engine)

    inserted, skipped = 0, 0

    #Using begin() context to automatically handle commit/rollback
    with engine.begin() as conn:
        for _, row in df.iterrows():
            stmt = insert(table).values(**row.to_dict())
            stmt = stmt.on_conflict_do_nothing(index_elements=[key_column])
            result = conn.execute(stmt)
            if result.rowcount > 0:
                inserted += 1
            else:
                skipped += 1

    print(f"{table_name}: {inserted} rows inserted, {skipped} skipped.")

In [63]:
print(dim_date.columns.tolist())


['date', 'year', 'month', 'day', 'quarter', 'weekday', 'date_id']


In [73]:
upsert_table(dim_date, 'dim_date','date', engine)
upsert_table(dim_customer, 'dim_customer', 'customer_id', engine)
upsert_table(dim_product, 'dim_product', 'product_id', engine)


dim_date: 0 rows inserted, 1434 skipped.
dim_customer: 0 rows inserted, 793 skipped.
dim_product: 0 rows inserted, 1862 skipped.


######  Populating fact table

In [92]:
def insert_fact_table(df, table_name, engine):
    metadata = sqlalchemy.MetaData()
    table = sqlalchemy.Table(table_name, metadata, autoload_with=engine)
    
    with engine.begin() as conn:
        for _, row in df.iterrows():
            stmt = insert(table).values(**row.to_dict())
            conn.execute(stmt)
    
    print(f"{table_name}: {len(df)} rows inserted.")


In [93]:
#Populate fact table
insert_fact_table(fact_sales, 'fact_sales', engine)

fact_sales: 9994 rows inserted.


###### Confirming operations

In [94]:
#checking fac_table
query = "SELECT * FROM fact_sales;"
df_fact = pd.read_sql(query, engine)
df_fact.head()

,fact_id,order_id,product_id,order_date_id,ship_date_id,customer_id,sales,quantity,discount,profit,revenue
0,1,CA-2016-152156,FUR-BO-10001798,1,584,CG-12520,261.9600,2,0.00,41.9136,523.9200
1,2,CA-2016-152156,FUR-CH-10000454,1,584,CG-12520,731.9400,3,0.00,219.5820,2195.8200
2,3,CA-2016-138688,OFF-LA-10000240,2,951,DV-13045,14.6200,2,0.00,6.8714,29.2400
3,4,US-2015-108966,FUR-TA-10000577,3,1091,SO-20335,957.5775,5,0.45,-383.0310,4787.8875
4,5,US-2015-108966,OFF-ST-10000760,3,1091,SO-20335,22.3680,2,0.20,2.5164,44.7360


In [95]:
#checking shape
shape_fact = df_fact.shape
print(shape_fact)

(9994, 11)


In [96]:
#checking dimension tables(date)
query = "SELECT * FROM dim_date;"
df_date = pd.read_sql(query, engine)
df_date.head()

,date_id,date,year,month,day,quarter,weekday
0,1,2016-11-08,2016,11,8,4,Tuesday
1,2,2016-06-12,2016,6,12,2,Sunday
2,3,2015-10-11,2015,10,11,4,Sunday
3,4,2014-06-09,2014,6,9,2,Monday
4,5,2017-04-15,2017,4,15,2,Saturday


In [97]:
df_date.shape

(1434, 7)

In [99]:
#Checking customer dimension
query = 'SELECT * FROM dim_customer';
df_cus = pd.read_sql(query, engine)
df_cus.head()

,customer_id,customer_name,segment,country,city,state,postal_code,region
0,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South
1,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West
2,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South
3,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West
4,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South


In [100]:
#shape
df_cus.shape

(793, 8)

In [101]:
#checking product dimension
query = 'SELECT * FROM dim_product';
df_pro = pd.read_sql(query, engine)
df_pro.head()

,product_id,product_name,category,sub_category
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage


In [104]:
#checking shape
df_pro.shape

(1862, 4)